In [4]:
import pandas as pd
from sqlalchemy import create_engine, text

# --- CONFIGURATION ---
DB_CONFIG = {
    'user': 'pgis',
    'password': 'pgis',
    'host': 'localhost',
    'port': '5437', 
    'database': 'MD'
}
FILE_PATH = "data_sensible/24 12 Mais.Droit STATS 2024.xlsx"
MOIS_SHEETS = ['Jan', 'Fev', 'Mar', 'Avr', 'Mai', 'Juin', 'Juil', 'Aoû', 'Sep', 'Oct', 'Nov', 'Déc']

def alimenter_bd_sans_doublons():
    engine = create_engine(f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}")
    print("--- DÉBUT DE L'ALIMENTATION (CORRECTION DES DOUBLONS) ---")

    for idx, sheet_name in enumerate(MOIS_SHEETS, 1):
        try:
            # 1. Lecture de l'onglet
            df = pd.read_excel(FILE_PATH, sheet_name=sheet_name)
            
            # 2. Nettoyage initial du N°
            df['N°'] = pd.to_numeric(df['N°'], errors='coerce')
            df = df.dropna(subset=['N°'])
            df['N°'] = df['N°'].astype(int)

            # --- CORRECTION CRUCIALE ---
            # Si un N° apparaît 2 fois dans le même onglet, on ne garde que la 1ère ligne
            nb_avant = len(df)
            df = df.drop_duplicates(subset=['N°'], keep='first')
            nb_apres = len(df)
            
            if nb_avant != nb_apres:
                print(f" > Onglet {sheet_name} : {nb_avant - nb_apres} doublon(s) supprimé(s) dans l'Excel.")

            # 3. Tri et nettoyage Partenaire
            df = df.sort_values(by='N°')
            if 'Partenaire' in df.columns:
                df['Partenaire'] = df['Partenaire'].astype(str).replace(['nan', 'NaN', 'None', '', ' '], None)
                df['Partenaire'] = df['Partenaire'].where(pd.notnull(df['Partenaire']), None)

            # 4. Suppression des données existantes en base (pour repartir à zéro sur ce mois)
            liste_ids = tuple(df['N°'].tolist())
            if liste_ids:
                # Gestion du cas où il n'y a qu'un seul ID (évite l'erreur SQL sur le format du tuple)
                condition_ids = f"({liste_ids[0]})" if len(liste_ids) == 1 else str(liste_ids)
                
                with engine.begin() as conn:
                    conn.execute(text(f"DELETE FROM demande WHERE num IN {condition_ids}"))
                    conn.execute(text(f"DELETE FROM solution WHERE num IN {condition_ids}"))
                    conn.execute(text(f"DELETE FROM entretien WHERE num IN {condition_ids}"))

            # 5. Insertion Entretien
            df_ent = pd.DataFrame()
            df_ent['num'] = df['N°']
            df_ent['date_ent'] = pd.to_datetime(f'2024-{idx:02d}-01')
            df_ent['mode'] = pd.to_numeric(df['Mode'], errors='coerce')
            df_ent['duree'] = pd.to_numeric(df['Durée'], errors='coerce')
            df_ent['sexe'] = pd.to_numeric(df['Sexe'], errors='coerce')
            df_ent['age'] = pd.to_numeric(df['Age'], errors='coerce')
            df_ent['vient_pr'] = pd.to_numeric(df['Vient pr'], errors='coerce')
            df_ent['sit_fam'] = df['Sit° Fam'].astype(str)
            df_ent['enfant'] = pd.to_numeric(df['Enfts'], errors='coerce')
            df_ent['modele_fam'] = pd.to_numeric(df['Modèle fam.'], errors='coerce')
            df_ent['profession'] = pd.to_numeric(df['Prof°'], errors='coerce')
            df_ent['ress'] = pd.to_numeric(df['Ress. 1'], errors='coerce')
            df_ent['origine'] = df['Origine'].astype(str)
            df_ent['commune'] = df['Domicile'].astype(str)
            df_ent['partenaire'] = df['Partenaire']

            df_ent.to_sql('entretien', engine, if_exists='append', index=False)

            # 6. Insertion Demande et Solution
            alimenter_table_liee(df, engine, 'demande', ['Dem.1', 'Dem.2', 'Dem.3'])
            alimenter_table_liee(df, engine, 'solution', ['Sol.1', 'Sol.2', 'Sol.3'])

            print(f" [OK] {sheet_name} traité avec succès.")

        except Exception as e:
            print(f" [ERREUR] {sheet_name} : {e}")

    # Synchronisation finale
    with engine.begin() as conn:
        conn.execute(text("SELECT setval('entretien_num_seq', (SELECT MAX(num) FROM entretien))"))
    print("\n--- TERMINÉ : TOUS LES MOIS SONT À JOUR ---")

def alimenter_table_liee(df_source, engine, table_nom, colonnes_excel):
    lignes = []
    for _, row in df_source.iterrows():
        for i, col in enumerate(colonnes_excel, 1):
            val = row.get(col)
            if pd.notna(val) and str(val).strip() not in ["", "nan", "NaN", "None"]:
                lignes.append({'num': int(row['N°']), 'pos': i, 'nature': str(val).strip()})
    if lignes:
        pd.DataFrame(lignes).to_sql(table_nom, engine, if_exists='append', index=False)

if __name__ == "__main__":
    alimenter_bd_sans_doublons()

--- DÉBUT DE L'ALIMENTATION (CORRECTION DES DOUBLONS) ---
 [OK] Jan traité avec succès.
 > Onglet Fev : 1 doublon(s) supprimé(s) dans l'Excel.
 [OK] Fev traité avec succès.
 [OK] Mar traité avec succès.
 [OK] Avr traité avec succès.
 [OK] Mai traité avec succès.
 [OK] Juin traité avec succès.
 > Onglet Juil : 1 doublon(s) supprimé(s) dans l'Excel.
 [OK] Juil traité avec succès.
 [OK] Aoû traité avec succès.
 > Onglet Sep : 1 doublon(s) supprimé(s) dans l'Excel.
 [OK] Sep traité avec succès.
 [OK] Oct traité avec succès.
 > Onglet Nov : 1 doublon(s) supprimé(s) dans l'Excel.
 [OK] Nov traité avec succès.
 [OK] Déc traité avec succès.

--- TERMINÉ : TOUS LES MOIS SONT À JOUR ---
